### IMPORT LIBRARY

In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.runtime.auth.client_credential import ClientCredential
from office365.runtime.client_request_exception import ClientRequestException
import datetime
import pytz
import pandas as pd
import numpy as np
import csv
import glob
import pyodbc
import os 
import json
from io import BytesIO
import io
import platform
from function.PyToSp import *
from tqdm import tqdm
from urllib.parse import quote_plus
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, event
import pyodbc
import requests
import inspect
from validate import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from urllib.parse import quote_plus
import msal
from itertools import chain
from send_email import *

### CONNECT TO AZURE SQL

In [2]:
import json
f = open ('database_information.json', "r")
qq = json.loads(f.read())
f.close()
ini_cnt_str ='Driver={driver_str};Server=tcp:hkazdevsqld3vnreserch.database.windows.net,1433;database={database};Uid={username};Pwd={password};Encrypt=yes;Authentication=ActiveDirectoryPassword;Connection Timeout=30;'.format(**qq)
quoted = quote_plus(ini_cnt_str)
cnt_str = 'mssql+pyodbc:///?odbc_connect={}'.format(quoted)
engine = create_engine(cnt_str)

### CONNECT TO SHAREPOINT

In [3]:
header_BIHub = 'share_point_BIHub'
config_BIHub = read_config_json(config_path, header_BIHub)
BIHub = SharePoint(config_BIHub)
BIHub.check_connect()

Web site title: S22M Research & BI Hub _ Successful Connection!


In [4]:
#Tất cả các tỉnh
relative_url = "/sites/BIHub/Shared Documents/Advisory Data/Macro commentary/Flat file"
sp_object = relative_url.split('/')[2].replace('-','')
list_folder = eval(sp_object).get_content_url(relative_url,return_list_folder=True)

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Macro commentary/Flat file/2022


In [5]:
#lấy năm
list_folder_sub1=[]
for i in list_folder:
    ls=eval(sp_object).get_content_url(i,return_list_folder=True)
    list_folder_sub1=list_folder_sub1+ls

Folder name: /sites/BIHub/Shared Documents/Advisory Data/Macro commentary/Flat file/2022/Q1
/sites/BIHub/Shared Documents/Advisory Data/Macro commentary/Flat file/2022/Q2


In [6]:
quarter = ['Q1']#Chọn năm cần import
year = ['2022']#Chọn quý cần import
#-------------------------------------------------------
df_summ_file = pd.DataFrame({'Name':[],'ServerRelativeUrl':[], 'TimeLastModified':[], 'ModTime':[], 'Modified_by_ID':[]})
for i in list_folder_sub1:
    if i.split('/')[-2] in year and i.split('/')[-1] in quarter:
        df_summ_file = pd.concat([df_summ_file, eval(sp_object).get_content_url(i)])
list_file = df_summ_file['ServerRelativeUrl'].to_list()
#History file
df_query=pd.DataFrame(df_summ_file).reset_index(drop=True)
df_summ_file = df_summ_file.reset_index(drop=True)

Folder name: 
Files name: /sites/BIHub/Shared Documents/Advisory Data/Macro commentary/Flat file/2022/Q1/Macro_20220331.xlsx


In [7]:
url = []
for i in list_file:
    data_key = i.split('/')[-1].split('.')[0].split('_')[-1]
    if str(data_key) not in ('x'):
        url.append(i)
url        

['/sites/BIHub/Shared Documents/Advisory Data/Macro commentary/Flat file/2022/Q1/Macro_20220331.xlsx']

### IMPORT DATA

In [8]:
'''Prepare ingredients'''
columns_that_need_unidecode=['City', 'Commentary_Image'
                            ]
#Create empty df for checking dictionary
df_dict = pd.DataFrame(columns=['File_Name', 'Missing_Values', 'Flag'])
#-------------------------------------------------------
'''Get data''' 
for file_url in tqdm(url):
    data, file_name, sector = get_data(relative_url, file_url)
    #-------------------------------------------------------
    data = check_date_key(file_url, data)#Check format date_key in flat file
    #-------------------------------------------------------
    '''Validation step'''
    #Remove unfortmated values
    data = remove_unformated_character(data)
    #Remove unicode characters
    for i in columns_that_need_unidecode:
        data[i] = remove_unicode(data[i])
    #Check dictionary
    lst_dict = ['City']
    lst_cls = ['City']
    for i, j in zip(lst_cls, lst_dict):
        data, df_dict = check_dictionary(df_dict, file_name, data, i, j, sector, engine, sp_object)
    if len(df_dict) == 0:
        print(colored('Validate succesfully','green'))
        data = Generate_Additional_Columns(data,url,df_summ_file,BIHub,engine,file_url)
        data.to_sql('Macro_Commentary', engine, index=False, if_exists='append', schema='Fresh')
    else:
        pass

  0%|          | 0/1 [00:01<?, ?it/s]

Can not read Macro_20220331.xlsx


AttributeError: 'bool' object has no attribute 'dropna'